主題：卡片配對記憶遊戲
===
介紹：
畫面上會有20張卡片，使用者依序選擇兩張卡片開啟，若卡片內容相同即完成配對，若卡片內容不同，於一秒之後將會覆蓋起來，直到所有的卡片配對完成，
遊戲結束後記錄使用者的點擊次數與時間。


實作說明：
* 透過numpy array儲存卡片資訊(正面、反面、是否開啟...)
* 透過numpy.random.shuffle打亂array的順序，讓卡片內容能隨機分布。
* 透過條件判斷，來改變卡片正反面的狀態及顯示。
* 透過Jupyter Widget中的Button呈現卡片
* 透過Jupyter Widget中的Layout控制卡片風格
* 透過Jupyter Widget中的Hbox, VBox進行排版 (使用numpy.reshape來產生想要的大小作為輸入參數)
* 於所有卡片配對完成後，透過time來計算總花費時間並統計點擊次數。
* 透過檔案記錄個別使用者遊玩紀錄

![image.png](https://github.com/YuChunTsao/PythonMLDL/blob/master/Midterm/demo1.png?raw=true)
![image.png](https://github.com/YuChunTsao/PythonMLDL/blob/master/Midterm/demo2.png?raw=true)

In [1]:
%matplotlib inline
import numpy as np
import time
import json
from ipywidgets import Button, Layout, HBox, VBox, Text
from IPython.display import clear_output

# user name
username = 'anonymous'

# 總時間
start_time = 0
end_time = 0
total_time = 0

# 點擊次數
click_counts = 0

# 完成的數量
finished = 0

# 所有的內容
content_list = ["(º﹃º )" ,"ఠ_ఠ" ,"(◕д◕✿)" ,"v(￣ｰ￣)v" ,"(`・ω・´)" ,"(￣∇￣)" ,"(´ﾟдﾟ`)" ,"(≧▽≦)" ,"｡^‿^｡" ,"(◕ᴥ◕)" ]

# 內容總數
counts = len(content_list)

# 比較兩張卡片是否相同的空間
temp_list = []

# 正面：編號
front = np.arange(counts*2)

# 背面：內容
back = np.array([content_list[i//2] for i in front])

# 開啟狀態
status = np.zeros(counts*2, dtype=bool)

# 打亂內容
np.random.shuffle(back)

# 將所有資訊放到cards中
cards = np.array(list(zip(front, back, status)))

# 排行榜
ranking_list = {
    "name": [],
    "time": [],
    "click_counts":[]
}

def selectCard(b):
    global temp_list
    
    # 該此選中的卡片編號
    index = int(b.description)
    button = Button(
        description = b.description,
        button_style = b.button_style,
        tooltip = b.tooltip,
        layout = b.layout
    )
    temp_list.append(button)
    b.description = back[index]
    b.button_style = 'warning'
    b.disabled = True

def successfulStyle(b):
    b.button_style = 'success'
    b.disabled = True
    
def checkFinished():
    global finished
    global counts
    global click_counts
    global start_time, end_time, total_time
    
    if finished == counts*2:
        end_time = time.time()
        total_time = end_time - start_time
        ranking_list["name"].append(username)
        ranking_list["time"].append(total_time)
        ranking_list["click_counts"].append(click_counts)
        
        # store to json file
        with open("ranking.json") as json_file:
            ranking_data = json.load(json_file)
            ranking_data["name"].append(username)
            ranking_data["time"].append(total_time)
            ranking_data["click_counts"].append(click_counts)      
            
        with open('ranking.json', 'w') as outfile:
            json.dump(ranking_data, outfile)
        
        print("花費：" + str(total_time) + "秒")
        print("點擊：" + str(click_counts) + "次")
        

def displayCard(b):
    global click_counts
    global finished
    global start_time
    global cards
    global items
    global temp_list
    
    if click_counts == 0:
        start_time = time.time()
    
    # 紀錄點擊次數
    click_counts+=1
    
    # 該此選中的卡片編號
    index = int(b.description)
    
    # 比較清單中無卡片
    if len(temp_list) == 0:
        # 加入此目標到清單中，並更新狀態。
        selectCard(b)

    # 比較清單已有一張卡片    
    elif len(temp_list) == 1:
        # 加入此目標到清單中，並更新狀態。
        selectCard(b)        
        card1_index = int(temp_list[0].description)
        card2_index = int(temp_list[1].description)

        # 判斷兩張卡片是否相同
        if back[card1_index] == back[card2_index]:
            successfulStyle(items[card1_index])
            successfulStyle(items[card2_index])       
            temp_list = []
            finished += 2
            checkFinished()          
        else:
            # 等待1秒給予使用者記憶
            time.sleep(1)
            items[card1_index].description = temp_list[0].description
            items[card1_index].button_style = ''
            items[card1_index].disabled = False
            
            items[card2_index].description = temp_list[1].description
            items[card2_index].button_style = ''   
            items[card2_index].disabled = False
            temp_list = []
            

def createButton(description):
    button_layout = Layout(
        width='100px',
        height='100px',
        border='solid'
    )    
    button = Button(
        description = str(description),
        button_style='',
        tooltip='Click Me!',
        layout = button_layout
    )
    
    button.on_click(displayCard)
    
    return button    

items = np.array([createButton(i) for i in front])
output = items.reshape(5, 4)                          


boxs = []
for i in output:
    item = i.tolist()
    box = VBox(item)
    boxs.append(box)
    
Cards = HBox(boxs)   

name_field = Text(
    value='',
    placeholder='Your name',
    description='Name:',
    disabled=False
)

def gamestart(e):
    clear_output(wait=True)
    
    global username
    name = name_field.value
    if name == '':
        name = 'anonymous'
    display(name_field)
    display(HBox([start, ranking]))   
    username = name
    print("Hi~" + name + "\n祝您遊戲愉快！")
    display(Cards)

start = Button(
    description='開始遊戲',
    button_style='success',
    tooltip='開始遊戲'
)
start.on_click(gamestart)


def findRanking(e):
    clear_output(wait=True)
    
    ranking_data = None

    with open("ranking.json") as json_file:
        ranking_data = json.load(json_file)
        
    display(name_field)
    display(HBox([start, ranking]))
    
    for i in range(0, len(ranking_data['name'])):
        if i == 0:
            print("%10s \t %10s \t %10s \n" % ("名稱", "時間", "點擊次數"))
        print("%10s \t %10.2f \t %10d " % (ranking_data['name'][i], ranking_data['time'][i], ranking_data['click_counts'][i]))
        print("\n")
        
    

ranking = Button(
    description='排行榜',
    button_style='success',
    tooltip='排行榜'
)
ranking.on_click(findRanking)


display(name_field)
HBox([start, ranking])

Text(value='', description='Name:', placeholder='Your name')

Hi~anonymous
祝您遊戲愉快！
